In [3]:
# !pip3 install dateparser
# !pip3 install lxml 

In [46]:
import requests
import pandas as pd
import dateparser
from datetime import datetime
from datetime import timedelta

In [5]:
r = requests.get('https://www.afl.com.au/news/564/afl-injury-list')
dfs = pd.read_html(r.text)

In [6]:
df = pd.DataFrame()
for df_part in dfs:
    updated_text = df_part.iloc[-1,0]
    df_part = df_part.iloc[:-1]
    date = updated_text[updated_text.find(':')+2:]
    df_part['Updated'] = date
    df = df.append(df_part)
df

<ipython-input-6-a9f2e973727b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_part['Updated'] = date


,PLAYER,INJURY,ESTIMATED RETURN,Updated
0,Jordon Butts,Ankle,Test,"Tuesday, March 16"
1,Brayden Cook,Shoulder,5 weeks,"Tuesday, March 16"
2,Matt Crouch,Groin,Test,"Tuesday, March 16"
3,Ben Davis,Ankle,2 weeks,"Tuesday, March 16"
4,Tom Doedee,Groin,Test,"Tuesday, March 16"
...,...,...,...,...
0,Hayden Crozier,Shoulder,6-8 weeks,"Tuesday, March 16"
1,Riley Garcia,Knee,TBC,"Tuesday, March 16"
2,Mitch Hannan,Groin,TBC,"Tuesday, March 16"
3,Toby McLean,Knee,4-5 months,"Tuesday, March 16"


In [7]:
df['DATE_BACK'] = df.apply(
    lambda x: 
    dateparser.parse(
        x['ESTIMATED RETURN'], 
        settings={
            'PREFER_DATES_FROM': 'future', 
            'PARSERS':['relative-time'], 
            'RELATIVE_BASE': dateparser.parse(x['Updated'])}, ), axis=1)

In [40]:
df['ESTIMATED RETURN'].unique()

array(['Test', '5 weeks', '2 weeks', '2-3 weeks', 'TBC', '2-5 weeks',
       'Season', 'Indefinite', '2-4 weeks', '6-8 weeks', '1 week',
       '4-5 weeks', '1-3 weeks', 'Round two', '4-8 weeks', '1-2 weeks',
       '3-4 weeks', '7 weeks', '5-6 months', 'TBA', '4 weeks', '3 weeks',
       '7-8 weeks', '12-16 weeks', '8-10 weeks', '4-6 weeks',
       'Round five', '4-5 months', '3-6 weeks', 'Round three',
       '6-10 weeks', '5-7 weeks', 'Round 23'], dtype=object)

In [50]:
df.loc[df['ESTIMATED RETURN'].isin([ 'Season', 'Indefinite']), 'DATE_BACK'] = datetime.strptime('01/01/24', '%d/%m/%y')
df.loc[df['ESTIMATED RETURN'].isin([ 'Season', 'Indefinite'])].loc[:, 'DATE_BACK']

df.loc[df['ESTIMATED RETURN'].isin([ 'Test', 'TBC', 'TBA']), 'DATE_BACK'] = datetime.now().date()+timedelta(days=28)
df.loc[df['ESTIMATED RETURN'].isin([ 'Test', 'TBC', 'TBA'])].loc[:, 'DATE_BACK']



0     2021-04-19
2     2021-04-19
4     2021-04-19
5     2021-04-19
0     2021-04-19
1     2021-04-19
2     2021-04-19
3     2021-04-19
1     2021-04-19
6     2021-04-19
8     2021-04-19
0     2021-04-19
3     2021-04-19
0     2021-04-19
1     2021-04-19
2     2021-04-19
4     2021-04-19
6     2021-04-19
7     2021-04-19
8     2021-04-19
13    2021-04-19
3     2021-04-19
2     2021-04-19
0     2021-04-19
2     2021-04-19
1     2021-04-19
3     2021-04-19
4     2021-04-19
5     2021-04-19
7     2021-04-19
0     2021-04-19
2     2021-04-19
4     2021-04-19
5     2021-04-19
6     2021-04-19
0     2021-04-19
3     2021-04-19
6     2021-04-19
7     2021-04-19
1     2021-04-19
3     2021-04-19
1     2021-04-19
2     2021-04-19
0     2021-04-19
1     2021-04-19
3     2021-04-19
9     2021-04-19
1     2021-04-19
2     2021-04-19
4     2021-04-19
Name: DATE_BACK, dtype: object

In [ ]:
df.to_csv('injuries.csv')